# Preparation

Load necessary modules and initialize relavent devices

In [1]:
%run seisidd/tomo_init.py

/home/beams/S6BM/opt/miniconda3/envs/bluesky_py37/lib/python3.7/site-packages/databroker/_core.py:962: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(f)
/home/beams/S6BM/opt/miniconda3/envs/bluesky_py37/lib/python3.7/site-packages/epics/pv.py:110: UserWarning: The `context` kwarg for epics.get_pv() is deprecated. New PVs will _not_ be created in the requested context.
  'The `context` kwarg for epics.get_pv() is deprecated. New PVs '


In [2]:
mode.set('dryrun')

In [3]:
A_shutter = get_shutter('dryrun')
tomostage = get_motors('dryrun') 
preci     = tomostage.preci              
samX      = tomostage.samX               
ksamX     = tomostage.ksamX
ksamZ     = tomostage.ksamZ        
samY      = tomostage.samY               
psofly    = get_fly_motor(mode='dryrun')
tomostage.psofly = psofly
det       = get_detector(mode='dryrun')

In [4]:
tomostage.preci

EpicsMotor(prefix='6bmpreci:m1', name='tomostage_preci', parent='tomostage', settle_time=0.0, timeout=None, read_attrs=['user_readback', 'user_setpoint'], configuration_attrs=['user_offset', 'user_offset_dir', 'velocity', 'acceleration', 'motor_egu'])

You can check the __prefined variables and functions__ in the following two dictionaries:

```
keywords_vars = {}  # {name: short description}
keywords_func = {}  # {name: short descciption}
```

or the corresponding functions:

```
list_predefined_vars()
list_predefined_func()
```

In [5]:
mode

Current runtime mode is set to: dryrun ['debug', 'dryrun', 'production']

In [6]:
print('func:')
list_predefined_func()

func:


name,description
get_runengine,Get a bluesky RunEngine
load_config,load yaml to a dict
get_shutter,Return a connection to a sim/real shutter
get_motors,Return a connection to sim/real tomostage motor
get_fly_motor,Return a connection to fly IOC control
get_detector,Return a connection to the detector
resume_motors_position,Move motors back to init position


__Fill__ out the __experiment details__ below for the metadata handler (MongoDB)

In [7]:
RE.md['beamline_id']      = 'APS 6-ID-D'
RE.md['proposal_id']      = 'internal test'
RE.md['pid']              = os.getpid()
RE.md['login_id']         = USERNAME + '@' + HOSTNAME
RE.md['BLUESKY_VERSION']  = bluesky.__version__
RE.md['OPHYD_VERSION']    = ophyd.__version__
RE.md['apstools_VERSION'] = apstools.__version__
RE.md['SESSION_STARTED']  = datetime.isoformat(datetime.now(), " ")

# Initialization

Perform control initialization for all related hardware.

In [8]:
mode.set('dryrun')

Install global __suspenders__ as a safeguard for this experiment

In [9]:
import apstools.devices      as APS_devices
from   bluesky.suspenders import SuspendFloor

aps = APS_devices.ApsMachineParametersDevice(name="APS")

# monitor ring current
suspend_APS_current = SuspendFloor(aps.current, 2, resume_thresh=10)
RE.install_suspender(suspend_APS_current)

# monitor shutter status
# NOTE: do not install right away since we need to close the shutter to collect dark field
suspend_A_shutter = SuspendFloor(A_shutter.pss_state, 1)


Import the __necessary__ predefined plan stubs

In [10]:
from seisidd.tomo_plans import tomo_scan

scan_cfg = 'seisidd/config/tomo_scan_template.yml'

Make sure check the scan with _summarize_plan_ before running

In [11]:
from bluesky.simulators import summarize_plan

In [12]:
summarize_plan(tomo_scan(det, tomostage, A_shutter, suspend_A_shutter, scan_cfg, init_motors_pos))

=================================== Open Run ===================================
A_shutter -> open
det_tiff1_file_path -> /dev/shm/tmp/
det_tiff1_file_name -> ttt
det_tiff1_file_write_mode -> 2
det_tiff1_num_capture -> 66
det_tiff1_file_template -> %s%s_%06d.hdf
det_hdf1_file_path -> /dev/shm/tmp/
det_hdf1_file_name -> ttt
det_hdf1_file_write_mode -> 2
det_hdf1_num_capture -> 66
det_hdf1_file_template -> %s%s_%06d.hdf
det_tiff1_enable -> 0
det_hdf1_enable -> 1
det_hdf1_capture -> 1
det_cam_frame_type -> 0
=================================== Open Run ===================================
tomostage_ksamX -> -1.0
tomostage_ksamZ -> 0.0
det_hdf1_nd_array_port -> PROC1
det_tiff1_nd_array_port -> PROC1
det_proc1_enable -> 1
det_proc1_reset_filter -> 1
det_proc1_num_filter -> 1
det_cam_trigger_mode -> Internal
det_cam_image_mode -> Multiple
det_cam_num_images -> 5
  Read ['det']
tomostage_samX -> 0.0
tomostage_samY -> 0.0
================================== Close Run ============================

In [13]:
RE(tomo_scan(det, tomostage, A_shutter, suspend_A_shutter, scan_cfg, init_motors_pos))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/beams/S6BM/opt/miniconda3/envs/bluesky_py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-f9f71f34607f>", line 1, in <module>
    RE(tomo_scan(det, tomostage, A_shutter, suspend_A_shutter, scan_cfg, init_motors_pos))
  File "/home/beams/S6BM/opt/miniconda3/envs/bluesky_py37/lib/python3.7/site-packages/bluesky/run_engine.py", line 742, in __call__
    self.loop.run_forever()
  File "/home/beams/S6BM/opt/miniconda3/envs/bluesky_py37/lib/python3.7/asyncio/base_events.py", line 526, in run_forever
    raise RuntimeError('This event loop is already running')
RuntimeError: This event loop is already running

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/beams/S6BM/opt/miniconda3/envs/bluesky_py37/lib/python3.7/site-packages/IPython/core/interactiveshell.p

RuntimeError: This event loop is already running

Transient Scan ID: 1     Time: 2019-08-05 16:56:25
Persistent Unique Scan ID: '172f8622-4402-4650-80b9-21a1726428ca'



[E 16:56:26.339 run_engine:1261] Run aborted
    Traceback (most recent call last):
      File "/home/beams/S6BM/opt/miniconda3/envs/bluesky_py37/lib/python3.7/site-packages/bluesky/run_engine.py", line 1150, in _run
        msg = self._plan_stack[-1].send(resp)
      File "/home/beams29/S6BM/opt/bluesky_test/jupyter-ht-hedm/seisidd/tomo_plans.py", line 216, in tomo_scan
        return (yield from scan_closure())
      File "/home/beams/S6BM/opt/miniconda3/envs/bluesky_py37/lib/python3.7/site-packages/bluesky/utils.py", line 1011, in dec_inner
        return (yield from plan)
      File "/home/beams/S6BM/opt/miniconda3/envs/bluesky_py37/lib/python3.7/site-packages/bluesky/preprocessors.py", line 939, in stage_wrapper
        return (yield from finalize_wrapper(inner(), unstage_devices()))
      File "/home/beams/S6BM/opt/miniconda3/envs/bluesky_py37/lib/python3.7/site

ERROR:bluesky.RE.139631694011136:Run aborted
Traceback (most recent call last):
  File "/home/beams/S6BM/opt/miniconda3/envs/bluesky_py37/lib/python3.7/site-packages/bluesky/run_engine.py", line 1150, in _run
    msg = self._plan_stack[-1].send(resp)
  File "/home/beams29/S6BM/opt/bluesky_test/jupyter-ht-hedm/seisidd/tomo_plans.py", line 216, in tomo_scan
    return (yield from scan_closure())
  File "/home/beams/S6BM/opt/miniconda3/envs/bluesky_py37/lib/python3.7/site-packages/bluesky/utils.py", line 1011, in dec_inner
    return (yield from plan)
  File "/home/beams/S6BM/opt/miniconda3/envs/bluesky_py37/lib/python3.7/site-packages/bluesky/preprocessors.py", line 939, in stage_wrapper
    return (yield from finalize_wrapper(inner(), unstage_devices()))
  File "/home/beams/S6BM/opt/miniconda3/envs/bluesky_py37/lib/python3.7/site-packages/bluesky/preprocessors.py", line 500, in finalize_wrapper
    ret = yield from plan
  File "/home/beams/S6BM/opt/miniconda3/envs/bluesky_py37/lib/pytho